In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import random

user_agents = [
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/59.0.3071.115 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_5) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/13.1.1 Safari/605.1.15",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:77.0) Gecko/20100101 Firefox/77.0",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.97 Safari/537.36",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.61 Safari/537.36",
    "Mozilla/5.0 (Windows NT 10.0; WOW64; Trident/7.0; rv:11.0) like Gecko",
    "Mozilla/5.0 (iPad; CPU OS 13_5 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/13.1 Mobile/15E148 Safari/604.1",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.61 Safari/537.36",
    "Mozilla/5.0 (iPhone; CPU iPhone OS 13_5 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/13.1.1 Mobile/15E148 Safari/604.1",
    "Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:77.0) Gecko/20100101 Firefox/77.0",
    "Mozilla/5.0 (Linux; Android 10; SM-G975F) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.96 Mobile Safari/537.36",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:68.0) Gecko/20100101 Firefox/68.0",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Edge/83.0.478.37",
    "Mozilla/5.0 (Linux; Android 10; SM-G973F) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.96 Mobile Safari/537.36"
]
def create_driver_with_user_agent(user_agent):
    options = Options()
    options.add_argument("--disable-dev-shm-usage")
    options.add_argument("--no-sandbox")
    options.add_argument(f'user-agent={user_agent}')
    return webdriver.Chrome(options=options)

In [2]:
import os

def get_file_names(folder_path):
    # List to hold file names
    file_names = []

    # Loop through all files in the folder
    for file in os.listdir(folder_path):
        if os.path.isfile(os.path.join(folder_path, file)):
            file_names.append(file)

    return file_names

# Replace this with the path to your folder
folder_path = 'ovpns/'

# Get the list of file names
ovpns = get_file_names(folder_path)
ovpns = [x.split(".")[0] for x in ovpns]

In [3]:
import subprocess

def nordvpn_connect(server):
    try:
        # Command to run the script as root with password from file
        command = f'sudo -S ./nordconnect.sh {server} < ovpns/term.txt'

        # Execute the command
        process = subprocess.Popen(command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)

        # Wait for the process to complete
        stdout, stderr = process.communicate()

    except Exception as e:
        print(f"An error occurred: {e}")
        
def nordvpn_disconnect():
    try:
        # Command to run the script as root with password from file
        command = f'sudo -S ./nordisconnect.sh < ovpns/term.txt'

        # Execute the command
        process = subprocess.Popen(command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)

        # Wait for the process to complete
        stdout, stderr = process.communicate()

    except Exception as e:
        print(f"An error occurred: {e}")

In [4]:
from selenium.webdriver.common.by import By
import time

def start_driver(current_vpn_index):
    c_index = current_vpn_index
    nordvpn_connect(ovpns[c_index])
    c_index += 1
    if c_index == len(ovpns):
        c_index = 0
    
    login_button1 = "/html/body/div[3]/div[1]/div/div[2]/div/div/div/div[2]/div/div/div/div[2]/div/div[2]"
    login_button2 = "/html/body/div/div/div/div[2]/div/div/div/div[1]/div[1]/div[2]/div/a[2]"
    login_button3 = "/html/body/div/div/div/div[2]/div/div/div/div[1]/div[1]/div[2]/div[2]/div/div[4]/div"

    driver = create_driver_with_user_agent(random.choice(user_agents))
    driver.get('https://www.threads.net/login')
    time.sleep(3)

    login_button1 = driver.find_element(By.XPATH, login_button1)
    login_button1.click()

    time.sleep(3)
    login_button2 = driver.find_element(By.XPATH, login_button2)
    login_button2.click()
    
    time.sleep(3)
    login_button3 = driver.find_element(By.XPATH, login_button3)
    login_button3.click()
    
    time.sleep(5)
    
    return c_index, driver

In [5]:
base_path = "./div/div/div/div[2]"

def parse_user(post):
    user_path = f"{base_path}/div[2]/div/div/span/div/div/a"
    user = post.find_element(By.XPATH,user_path)
    return user.get_attribute('href')

def parse_caption(post):
    child_elements = post.find_elements(By.XPATH,f"{base_path}/child::*")
    number_of_children = len(child_elements) - 1 #Non conto i pulsanti
    if number_of_children < 4:
        caption_path = f"{base_path}/div[3]/div/div[1]/span"  
    else:
        caption_path = f"{base_path}/div[4]/div/div[1]/span"
    caption = post.find_element(By.XPATH,caption_path)
    try:
        caption_a = post.find_element(By.XPATH,f"{caption_path}/a")
        caption_url = caption_a.get_attribute('href')
        new_caption_text = caption.text.replace(caption_a.text, caption_url)
        return number_of_children,new_caption_text
    except:
        return number_of_children,caption.text

def parse_image(post, number_of_children, has_text=True):
    img_index = 2
    if not has_text:
        img_index = 1
    try:
        if number_of_children < 4:
            image_path = f"{base_path}/div[3]/div/div[{img_index}]/div/div/picture/img"
        else:
            image_path = f"{base_path}/div[4]/div/div[{img_index}]/div/div/picture/img"
        image = post.find_element(By.XPATH,image_path)
        image_url = image.get_attribute('src')
        return image_url
    except:
        return ""
    
def parse_post_url(post):
    try:
        url_path = f"{base_path}/div[2]/div/div[2]/span/a"
        url = post.find_element(By.XPATH,url_path)
        return url.get_attribute('href')
    except:
        return ""

In [6]:
import mysql.connector

def insert_post(post_url, user, image, caption):
    db_params = {
        'host': '192.168.100.200',  # Or your database host
        'user': 'ecorradini',   # Your database username
        'password': 'birillo',
        'database': 'threads'
    }
    db_connection = mysql.connector.connect(**db_params)
    db_cursor = db_connection.cursor()
    
    c = caption.replace('"','\"').replace("'","\'")
    insert_query = f"INSERT IGNORE INTO post (user_url, post_url, image_url, caption) VALUES (%s, %s, %s, %s)"
    
    data = (user,post_url,image,c)
    db_cursor.execute(insert_query, data)
    db_connection.commit()
    
    db_cursor.close()
    db_connection.close()

In [7]:
from selenium.webdriver.common.by import By
import time

def download(height, driver, current_vpn_index):
    c_driver = driver
    c_vpn = current_vpn_index
    
    time.sleep(4)
    posts_container_path = "/html/body/div/div/div/div[2]/div/div/div/div/div/div"
    posts_container = c_driver.find_element(By.XPATH,posts_container_path)
    posts = posts_container.find_elements(By.XPATH,"./div[position() >= 2]")
    for post in posts:
        try:
            user = parse_user(post)
            number_of_children,caption = parse_caption(post)
            has_caption = True
            if len(caption) == 0:
                has_caption = False
            image = parse_image(post,number_of_children,has_caption)
            url = parse_post_url(post)
        except:
            continue
        if len(url) > 0 and len(user) > 0:
            insert_post(url,user,image,caption)
            
    c_driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(4)
    new_height = c_driver.execute_script("return document.body.scrollHeight")
    if new_height == height:
        c_driver.close()
        nordvpn_disconnect()
        c_vpn2, c_driver2 = start_driver(c_vpn)
        time.sleep(8)
        current_height = c_driver2.execute_script("return document.body.scrollHeight")
        download(current_height, c_driver2, c_vpn2)
    else:
        download(new_height, c_driver, c_vpn)

current_vpn_index = 0
current_vpn_index, driver = start_driver(current_vpn_index)
current_height = driver.execute_script("return document.body.scrollHeight")
download(current_height, driver, current_vpn_index)

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/div[3]/div[1]/div/div[2]/div/div/div/div[2]/div/div/div/div[2]/div/div[2]"}
  (Session info: chrome=122.0.6261.57); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
#0 0x58b9c7bb9f33 <unknown>
#1 0x58b9c78b1ce6 <unknown>
#2 0x58b9c78fce48 <unknown>
#3 0x58b9c78fcf01 <unknown>
#4 0x58b9c79403f4 <unknown>
#5 0x58b9c791eedd <unknown>
#6 0x58b9c793d899 <unknown>
#7 0x58b9c791ec53 <unknown>
#8 0x58b9c78efdb3 <unknown>
#9 0x58b9c78f077e <unknown>
#10 0x58b9c7b7f86b <unknown>
#11 0x58b9c7b83885 <unknown>
#12 0x58b9c7b6d181 <unknown>
#13 0x58b9c7b84412 <unknown>
#14 0x58b9c7b5125f <unknown>
#15 0x58b9c7ba8528 <unknown>
#16 0x58b9c7ba8723 <unknown>
#17 0x58b9c7bb90e4 <unknown>
#18 0x7785ba294ac3 <unknown>
